# Database Access with Relational and Non-Relational Databases in Python




In [2]:
import sqlite3
import os

## 1. Relational Database: SQLite

SQLite is a relational database management system (RDBMS) that stores data in tables with rows and columns. It enforces schemas and supports SQL queries.

### Key Operations:
- Connect to/create database
- Create table
- Insert data
- Query data
- Update/Delete
- Close connection

In [3]:
# Connect to SQLite database (creates 'example.db' if it doesn't exist)
sqlite_db = 'example.db'
conn = sqlite3.connect(sqlite_db)
cursor = conn.cursor()

# Create a table
cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    age INTEGER,
    city TEXT
)
''')
conn.commit()
print('Table created.')

Table created.


In [4]:
# Insert data
users_data = [
    ('Alice', 25, 'New York'),
    ('Bob', 30, 'Los Angeles'),
    ('Charlie', 35, 'Chicago')
]
cursor.executemany('INSERT INTO users (name, age, city) VALUES (?, ?, ?)', users_data)
conn.commit()
print('Data inserted.')

Data inserted.


In [5]:
# Query data
cursor.execute('SELECT * FROM users')
rows = cursor.fetchall()
print('Query results:')
for row in rows:
    print(row)

# Query with condition
cursor.execute('SELECT * FROM users WHERE age > 28')
filtered_rows = cursor.fetchall()
print('\nFiltered results (age > 28):')
for row in filtered_rows:
    print(row)
    
# Query with logical operators
cursor.execute('SELECT * FROM users WHERE city = "New York" AND age <= 30')
filtered_rows = cursor.fetchall()
print('\nFiltered results (city is New York AND age <= 30):')
for row in filtered_rows:
    print(row)
    
# Query with bindnings. Bindings help prevent SQL injection attacks and improve performance.
age_limit = 30
cursor.execute('SELECT * FROM users WHERE age < ?', (age_limit,))
bound_rows = cursor.fetchall()
print(f'\nBound query results (age < {age_limit}):')
for row in bound_rows:
    print(row)


Query results:
(1, 'Alice', 25, 'New York')
(2, 'Bob', 30, 'Los Angeles')
(3, 'Charlie', 35, 'Chicago')

Filtered results (age > 28):
(2, 'Bob', 30, 'Los Angeles')
(3, 'Charlie', 35, 'Chicago')

Filtered results (city is New York AND age <= 30):
(1, 'Alice', 25, 'New York')

Bound query results (age < 30):
(1, 'Alice', 25, 'New York')


In [6]:
# Update data
cursor.execute('UPDATE users SET age = 26 WHERE name = "Alice"')
conn.commit()

# Delete data
cursor.execute('DELETE FROM users WHERE name = "Charlie"')
conn.commit()

# Verify changes
cursor.execute('SELECT * FROM users')
updated_rows = cursor.fetchall()
print('Updated table:')
for row in updated_rows:
    print(row)

Updated table:
(1, 'Alice', 26, 'New York')
(2, 'Bob', 30, 'Los Angeles')


In [7]:
# Close connection
conn.close()
print('SQLite connection closed.')

SQLite connection closed.


### Using SQLAlchemy for Relational Databases

SqAlchemy is an ORM (Object Relational Mapper) for Python that allows developers to interact with databases using Python objects instead of writing raw SQL queries. 

It provides a high-level abstraction over database operations, making it easier to work with relational databases in a more Pythonic way.

ORM helps in preventing SQL injection by using parameterized queries and abstractions that separate SQL code from data.

Sqlachemy ORM is different from binding queries that orm use object-oriented approach to interact with the database.


In [8]:
# pip install sqlalchemy


In [9]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Setup
engine = create_engine(f'sqlite:///{sqlite_db}')
Base = declarative_base()

# Define model
class User(Base):
    __tablename__ = 'users_orm'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    age = Column(Integer)
    city = Column(String)

# Create table
Base.metadata.create_all(engine)

# Session
Session = sessionmaker(bind=engine)
session = Session()

# Insert
new_user = User(name='David', age=40, city='Seattle')
session.add(new_user)
session.commit()

# Query
users = session.query(User).all()
print('ORM Query results:')
for user in users:
    print(f'{user.name}, {user.age}, {user.city}')

session.close()

/tmp/ipykernel_16424/4024939245.py:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


ORM Query results:
David, 40, Seattle


In [10]:
# Orm showing oprators 

## 2. Non-Relational Database: MongoDB

MongoDB is a NoSQL database that stores data in flexible, JSON-like documents. It doesn't enforce schemas and is great for unstructured data.

### Key Operations:
- Connect to database
- Create collection
- Insert documents
- Query documents
- Update/Delete
- Close connection

**Note**: Ensure MongoDB is running. If using Atlas, replace the connection string.

In [11]:
# pip install pymongo

from pymongo import MongoClient

In [12]:
# Connect to MongoDB (local default)
client = MongoClient('mongodb://localhost:27017/')
db = client['example_db']
collection = db['users']

# Drop collection if exists (for clean demo)
collection.drop()
print('Connected to MongoDB.')

Connected to MongoDB.


In [13]:
# Insert documents
users_docs = [
    {'name': 'Alice', 'age': 25, 'city': 'New York', 'hobbies': ['reading', 'travel']},
    {'name': 'Bob', 'age': 30, 'city': 'Los Angeles'},
    {'name': 'Charlie', 'age': 35, 'city': 'Chicago', 'hobbies': ['sports']}
]
insert_result = collection.insert_many(users_docs)
print('Documents inserted:', insert_result.inserted_ids)

Documents inserted: [ObjectId('69758cb71cef006dcdea6b90'), ObjectId('69758cb71cef006dcdea6b91'), ObjectId('69758cb71cef006dcdea6b92')]


### Comparison Operators 

| Operator | Meaning               | Example                 |
| -------- | --------------------- | ----------------------- |
| `$gt`    | Greater than          | `{'age': {'$gt': 28}}`  |
| `$lt`    | Less than             | `{'age': {'$lt': 28}}`  |
| `$gte`   | Greater than or equal | `{'age': {'$gte': 28}}` |
| `$lte`   | Less than or equal    | `{'age': {'$lte': 28}}` |
| `$eq`    | Equal                 | `{'age': {'$eq': 28}}`  |
| `$ne`    | Not equal             | `{'age': {'$ne': 28}}`  |

### Logical Operators

| Operator | Meaning       | Example                                                   |
| -------- | ------------- | --------------------------------------------------------- |
| `$and`   | AND condition | `{'$and': [{'age': {'$gt': 20}}, {'city': 'Kathmandu'}]}` |
| `$or`    | OR condition  | `{'$or': [{'age': {'$gt': 28}}, {'city': 'Pokhara'}]}`    |
| `$not`   | NOT condition | `{'age': {'$not': {'$gt': 28}}}`                          |
| `$nor`   | NOR (NOT OR)  | `{'$nor': [{'age': {'$lt': 18}}, {'age': {'$gt': 60}}]}`  |


In [14]:
# Query all documents
docs = list(collection.find())
print('All documents:')
for doc in docs:
    print(doc)

# Query with filter
filtered_docs = list(collection.find({'age': {'$gt': 28}}))
print('\nFiltered documents (age > 28):')
for doc in filtered_docs:
    print(doc)
    
# Query with logical operators
filtered_docs = list(collection.find({'$and': [{'city': 'New York'}, {'age': {'$lte': 30}}]}))
print('\nFiltered documents (city is New York AND age <= 30):')
for doc in filtered_docs:
    print(doc)



All documents:
{'_id': ObjectId('69758cb71cef006dcdea6b90'), 'name': 'Alice', 'age': 25, 'city': 'New York', 'hobbies': ['reading', 'travel']}
{'_id': ObjectId('69758cb71cef006dcdea6b91'), 'name': 'Bob', 'age': 30, 'city': 'Los Angeles'}
{'_id': ObjectId('69758cb71cef006dcdea6b92'), 'name': 'Charlie', 'age': 35, 'city': 'Chicago', 'hobbies': ['sports']}

Filtered documents (age > 28):
{'_id': ObjectId('69758cb71cef006dcdea6b91'), 'name': 'Bob', 'age': 30, 'city': 'Los Angeles'}
{'_id': ObjectId('69758cb71cef006dcdea6b92'), 'name': 'Charlie', 'age': 35, 'city': 'Chicago', 'hobbies': ['sports']}

Filtered documents (city is New York AND age <= 30):
{'_id': ObjectId('69758cb71cef006dcdea6b90'), 'name': 'Alice', 'age': 25, 'city': 'New York', 'hobbies': ['reading', 'travel']}


In [15]:
# Update document
collection.update_one({'name': 'Alice'}, {'$set': {'age': 26}})

# Delete document
collection.delete_one({'name': 'Charlie'})

# Verify changes
updated_docs = list(collection.find())
print('Updated collection:')
for doc in updated_docs:
    print(doc)

Updated collection:
{'_id': ObjectId('69758cb71cef006dcdea6b90'), 'name': 'Alice', 'age': 26, 'city': 'New York', 'hobbies': ['reading', 'travel']}
{'_id': ObjectId('69758cb71cef006dcdea6b91'), 'name': 'Bob', 'age': 30, 'city': 'Los Angeles'}


In [16]:
# Close connection
client.close()
print('MongoDB connection closed.')

MongoDB connection closed.


## Cleanup
Remove the SQLite database file.

In [17]:
# if os.path.exists(sqlite_db):
#     os.remove(sqlite_db)
#     print(f'Removed: {sqlite_db}')